<a href="https://colab.research.google.com/github/uroojfatima18/AI-Agents/blob/main/Tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import os

from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig
from google.colab import userdata

In [4]:
from agents import(
    Agent,
    Runner,
    set_default_openai_api,
    set_default_openai_client,
    set_tracing_disabled,
)

BASE_URL = os.getenv("EXAMPLE_BASE_URL") or "https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY = os.getenv("EXAMPLE_API_KEY") or userdata.get("GEMINI_API_KEY")
MODEL_NAME = os.getenv("EXAMPLE_MODEL_NAME") or "gemini-2.0-flash"

if not BASE_URL or not API_KEY or not MODEL_NAME:
    raise ValueError(
        "Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code."
    )

client=AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
)

set_default_openai_client(client=client, use_for_tracing=True)
set_default_openai_api("chat_completions")

In [5]:
import os
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, set_default_openai_api, set_default_openai_client, set_trace_processors
from agents.tracing.processor_interface import TracingProcessor
from pprint import pprint

class LocalTraceProcessor(TracingProcessor):
  def __init__(self):
    self.traces=[]
    self.spans=[]


  def on_trace_start(self, trace):
    self.traces.append(trace)
    print("trace_start",{trace.trace_id})

  def on_trace_end(self, trace):
    print("trace_end",{trace.export})

  def on_span_start(self, span):
        self.spans.append(span)
        print(f"Span started: {span.span_id}")
        print(f"Span details: ")
        pprint(span.export())

  def on_span_end(self, span):
        print(f"Span ended: {span.span_id}")
        print(f"Span details:")
        pprint(span.export())

  def force_flush(self):
        print("Forcing flush of trace data")


  def shutdown(self):
        print("=======Shutting down trace processor========")
        # Print all collected trace and span data
        print("Collected Traces:")
        for trace in self.traces:
            print(trace.export())
        print("Collected Spans:")
        for span in self.spans:
            print(span.export())


if not BASE_URL or not API_KEY or not MODEL_NAME:
    raise ValueError("Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code.")

# Create OpenAI client
client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
)

# Configure the client
set_default_openai_client(client=client, use_for_tracing=True)
set_default_openai_api("chat_completions")

In [6]:
# Set up the custom trace processor
local_processor = LocalTraceProcessor()
set_trace_processors([local_processor])

In [7]:

# Example function to run an agent and collect traces
async def main():
    agent = Agent(name="Example Agent", instructions="Perform example tasks.", model=MODEL_NAME)

    with trace("Example workflow"):
        first_result = await Runner.run(agent, "Start the task")
        second_result = await Runner.run(agent, f"Rate this result: {first_result.final_output}")
        print(f"Result: {first_result.final_output}")
        print(f"Rating: {second_result.final_output}")

# Run the main function
import asyncio
asyncio.run(main())

trace_start {'trace_02f3b587cf484b9dab9c39a8ada23d7e'}
Span started: span_e94151ffc2ef460aa24f5b28
Span details: 
{'ended_at': None,
 'error': None,
 'id': 'span_e94151ffc2ef460aa24f5b28',
 'object': 'trace.span',
 'parent_id': None,
 'span_data': {'handoffs': [],
               'name': 'Example Agent',
               'output_type': 'str',
               'tools': None,
               'type': 'agent'},
 'started_at': '2025-05-27T19:02:59.570375+00:00',
 'trace_id': 'trace_02f3b587cf484b9dab9c39a8ada23d7e'}
Span started: span_5e2429edb79f4b678adb2b98
Span details: 
{'ended_at': None,
 'error': None,
 'id': 'span_5e2429edb79f4b678adb2b98',
 'object': 'trace.span',
 'parent_id': 'span_e94151ffc2ef460aa24f5b28',
 'span_data': {'input': None,
               'model': 'gemini-2.0-flash',
               'model_config': {'base_url': 'https://generativelanguage.googleapis.com/v1beta/openai/',
                                'extra_body': None,
                                'extra_headers': None

In [9]:
api_key="fa56d1a5-1b65-45d4-b2b3-a8c0fe444f2f"

import agentops
# Initialize AgentOps



In [10]:
import asyncio
async def main():
    agentops.init(api_key)
    agent = Agent(name="Example Agent", instructions="Perform example tasks.", model=MODEL_NAME)

    with trace("Example workflow"):
        first_result = await Runner.run(agent, "Start the task")
        second_result = await Runner.run(agent, f"Rate this result: {first_result.final_output}")
        print(f"Result: {first_result.final_output}")
        print(f"Rating: {second_result.final_output}")

# Run the main function

asyncio.run(main())

🖇 AgentOps: Session Replay: https://app.agentops.ai/sessions?trace_id=a1e7e583690353b5772bdc8b83df7cff


Result: Okay, I'm ready.  What is the first task you'd like me to perform? Please be as specific as possible with your instructions. For example:

*   "Translate the following sentence into Spanish: 'Hello, how are you today?'"
*   "Summarize this article: [insert article text here]"
*   "Write a short poem about a cat."
*   "Calculate 15% of 250."

I'm looking forward to helping you!

Rating: This is a good, helpful response. Here's a breakdown:

*   **"Okay, I'm ready."** This signals that the model is active and listening.
*   **"What is the first task you'd like me to perform?"** Clear and direct question, indicating readiness to receive instructions.
*   **"Please be as specific as possible with your instructions."**  Very important! This encourages the user to provide clear and unambiguous instructions, which leads to better results.
*   **Example Instructions:** Providing example instructions is excellent. It gives the user a clear idea of the type of tasks the model can perform